In [39]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
import lightgbm as lgb

In [42]:
def create_optimized_scorer(preprocessor):
    """
    Optimized scorer function with parallel processing
    """
    def scorer(model_name, model):
        output = []
        output.append(model_name)
        
        # Set n_jobs for models that support it
        if hasattr(model, 'n_jobs'):
            model.set_params(n_jobs=-1)
        
        pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('regressor', model)
        ])
        
        # K-fold cross-validation with parallel jobs
        kfold = KFold(n_splits=10, shuffle=True, random_state=42)
        scores = cross_val_score(
            pipeline, X, y_transformed, 
            cv=kfold, 
            scoring='r2',
            n_jobs=-1  # Parallel cross-validation
        )
        
        output.append(scores.mean())
        
        X_train, X_test, y_train, y_test = train_test_split(
            X, y_transformed, test_size=0.2, random_state=42
        )
        print("Training the pipeline")
       
        pipeline.fit(X_train, y_train)
        print("predicting the pipeline")
        y_pred = pipeline.predict(X_test)
        y_pred = np.expm1(y_pred)
        print("appending the solution of pipeline")
        output.append(mean_absolute_error(np.expm1(y_test), y_pred))
        
        return output
    
    return scorer

In [43]:
def create_optimized_models():
    """
    Create models with optimized parameters for faster training
    """
    return {
        'linear_reg': LinearRegression(),
        'ridge': Ridge(),
        'LASSO': Lasso(),
        'svr': SVR(),  # SVR is slow, consider reducing C or using kernel='linear'
        'decision tree': DecisionTreeRegressor(max_depth=20),  # Limit depth
        
        # Optimized Random Forest
        'random forest': RandomForestRegressor(
            n_estimators=150,  # Reduced from default
            max_depth=20,      # Limit depth
            min_samples_split=10,  # Increase for speed
            min_samples_leaf=4,    # Increase for speed
            n_jobs=-1,         # Parallel processing
            random_state=42
        ),
        
        # Optimized Extra Trees
        'extra trees': ExtraTreesRegressor(
            n_estimators=100,
            max_depth=20,
            min_samples_split=10,
            min_samples_leaf=4,
            n_jobs=-1,
            random_state=42
        ),
        
        # Optimized Gradient Boosting
        'gradient boosting': GradientBoostingRegressor(
         
        ),
        
        'adaboost': AdaBoostRegressor(
            n_estimators=50,   # Reduced from default
            learning_rate=1.0,
            random_state=42
        ),
        
        'mlp': MLPRegressor(
            hidden_layer_sizes=(100,),  # Simpler architecture
            max_iter=200,      # Reduced iterations
            early_stopping=True,  # Stop early if no improvement
            random_state=42
        ),
        
        # Optimized XGBoost
        'xgboost': XGBRegressor(
         
        ),
        
        # Optimized LightGBM (fastest gradient boosting)
        'lightgbm': lgb.LGBMRegressor(
            
            n_jobs=-1,
            random_state=42,
            verbose=-1  # Suppress warnings
        )
    }





In [44]:
# ============================================
# OPTIMIZATION 3: Optimized preprocessing
# ============================================

def create_optimized_preprocessor(columns_to_encode):
    """
    Create preprocessor with handle_unknown for OrdinalEncoder
    """
    return ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), 
             ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
            ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), 
             columns_to_encode),
            ('cat1', OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False), 
             ['sector'])
        ], 
        remainder='passthrough',
        n_jobs=-1  # Parallel preprocessing
    )

In [45]:
def quick_model_selection(X, y_transformed, preprocessor):
    """
    Quick model selection - test only fast models first
    """
    print("Phase 1: Testing fast models...")
    
    fast_models = {
        'linear_reg': LinearRegression(),
        'ridge': Ridge(),
        'LASSO': Lasso(),
        'decision tree': DecisionTreeRegressor(max_depth=20),
        'xgboost': XGBRegressor(n_estimators=100, max_depth=6, n_jobs=-1, random_state=42),
        'lightgbm': lgb.LGBMRegressor(n_estimators=100, max_depth=6, n_jobs=-1, 
                                      random_state=42, verbose=-1)
    }
    
    scorer = create_optimized_scorer(preprocessor)
    fast_results = []
    
    for model_name, model in fast_models.items():
        print(f"  Training {model_name}...")
        fast_results.append(scorer(model_name, model))
    
    fast_df = pd.DataFrame(fast_results, columns=['name', 'r2', 'mae'])
    print("\nFast models results:")
    print(fast_df.sort_values(['mae']))
    
    return fast_df


def full_model_selection(X, y_transformed, preprocessor):
    """
    Full model selection including slower models
    """
    print("\nPhase 2: Testing all models...")
    
    all_models = create_optimized_models()
    scorer = create_optimized_scorer(preprocessor)
    all_results = []
    
    for model_name, model in all_models.items():
        print(f"  Training {model_name}...")
        all_results.append(scorer(model_name, model))
    
    full_df = pd.DataFrame(all_results, columns=['name', 'r2', 'mae'])
    print("\nAll models results:")
    print(full_df.sort_values(['mae']))
    
    return full_df


In [46]:
def quick_test_on_sample(X, y_transformed, preprocessor, sample_size=0.3):
    """
    Quick test on a sample of data to identify best models
    """
    print(f"Quick test on {int(sample_size*100)}% sample...")
    
    # Sample data
    X_sample = X.sample(frac=sample_size, random_state=42)
    y_sample = y_transformed[X_sample.index]
    
    models = create_optimized_models()
    scorer = create_optimized_scorer(preprocessor)
    
    results = []
    for model_name, model in models.items():
        print(f"  Testing {model_name}...")
        
        pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('regressor', model)
        ])
        
        # Single train-test split (faster than cross-validation)
        X_train, X_test, y_train, y_test = train_test_split(
            X_sample, y_sample, test_size=0.2, random_state=42
        )
        
        pipeline.fit(X_train, y_train)
        score = pipeline.score(X_test, y_test)
        
        y_pred = np.expm1(pipeline.predict(X_test))
        mae = mean_absolute_error(np.expm1(y_test), y_pred)
        
        results.append([model_name, score, mae])
    
    sample_df = pd.DataFrame(results, columns=['name', 'r2', 'mae'])
    print("\nQuick sample results:")
    print(sample_df.sort_values(['mae']))
    
    return sample_df

In [47]:
columns_to_encode = ['property_type', 'balcony', 'agePossession', 
                     'furnishing_type', 'luxury_category', 'floor_category']

In [48]:
df = pd.read_csv('concatenated_properties_for prediction.csv')

In [49]:
# 0 -> unfurnished
# 1 -> semifurnished
# 2 -> furnished
df['furnishing_type'] = df['furnishing_type'].replace({0.0:'unfurnished',1.0:'semifurnished',2.0:'furnished'})

In [50]:
X = df.drop(columns=['price'])
y = df['price']

In [51]:
# Applying the log1p transformation to the target variable
y_transformed = np.log1p(y)

In [52]:
preprocessor = create_optimized_preprocessor(columns_to_encode)

In [53]:
sample_results = quick_test_on_sample(X, y_transformed, preprocessor, sample_size=0.3)

Quick test on 30% sample...
  Testing linear_reg...
  Testing ridge...
  Testing LASSO...
  Testing svr...
  Testing decision tree...
  Testing random forest...
  Testing extra trees...
  Testing gradient boosting...
  Testing adaboost...
  Testing mlp...
  Testing xgboost...
  Testing lightgbm...

Quick sample results:
                 name        r2       mae
10            xgboost  0.810507  0.760677
7   gradient boosting  0.779517  0.849169
3                 svr  0.779563  0.865169
9                 mlp  0.750291  0.878099
6         extra trees  0.740019  0.903042
5       random forest  0.732514  0.931570
11           lightgbm  0.719073  0.980582
4       decision tree  0.672883  1.084246
0          linear_reg  0.679700  1.114385
1               ridge  0.694364  1.153454
8            adaboost  0.540370  1.287900
2               LASSO -0.000271  1.705427


In [54]:
fast_results = quick_model_selection(X, y_transformed, preprocessor)

Phase 1: Testing fast models...
  Training linear_reg...
Training the pipeline
predicting the pipeline
appending the solution of pipeline
  Training ridge...
Training the pipeline
predicting the pipeline
appending the solution of pipeline
  Training LASSO...
Training the pipeline
predicting the pipeline
appending the solution of pipeline
  Training decision tree...
Training the pipeline
predicting the pipeline
appending the solution of pipeline
  Training xgboost...
Training the pipeline
predicting the pipeline
appending the solution of pipeline
  Training lightgbm...
Training the pipeline
predicting the pipeline
appending the solution of pipeline

Fast models results:
            name        r2       mae
4        xgboost  0.826413  0.746002
5       lightgbm  0.812146  0.814708
3  decision tree  0.710579  0.858151
1          ridge  0.753087  0.964438
0     linear_reg  0.743327  0.969953
2          LASSO -0.002390  1.692700


In [55]:
full_results = full_model_selection(X, y_transformed, preprocessor)


Phase 2: Testing all models...
  Training linear_reg...
Training the pipeline
predicting the pipeline
appending the solution of pipeline
  Training ridge...
Training the pipeline
predicting the pipeline
appending the solution of pipeline
  Training LASSO...
Training the pipeline
predicting the pipeline
appending the solution of pipeline
  Training svr...
Training the pipeline
predicting the pipeline
appending the solution of pipeline
  Training decision tree...
Training the pipeline
predicting the pipeline
appending the solution of pipeline
  Training random forest...
Training the pipeline
predicting the pipeline
appending the solution of pipeline
  Training extra trees...
Training the pipeline
predicting the pipeline
appending the solution of pipeline
  Training gradient boosting...
Training the pipeline
predicting the pipeline
appending the solution of pipeline
  Training adaboost...
Training the pipeline
predicting the pipeline
appending the solution of pipeline
  Training mlp...
T

In [60]:
def analyze_n_estimators_impact(X, y_transformed, preprocessor, 
                                 estimator_range=[50, 100, 200, 300, 500]):
    """
    Analyze the impact of number of trees on performance and training time
    Shows diminishing returns of adding more trees
    """
    import time
    
    print("Analyzing n_estimators impact on RandomForest...")
    print("-" * 70)
    
    results = []
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_transformed, test_size=0.2, random_state=42
    )
    
    for n_est in estimator_range:
        start_time = time.time()
        
        model = RandomForestRegressor(
            n_estimators=n_est,
            max_depth=20,
            n_jobs=-1,
            random_state=42
        )
        
        pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('regressor', model)
        ])
        
        # Train and evaluate
        pipeline.fit(X_train, y_train)
        train_time = time.time() - start_time
        
        # Get scores
        r2_score = pipeline.score(X_test, y_test)
        y_pred = np.expm1(pipeline.predict(X_test))
        mae = mean_absolute_error(np.expm1(y_test), y_pred)
        
        results.append({
            'n_estimators': n_est,
            'r2_score': r2_score,
            'mae': mae,
            'train_time_sec': train_time,
            'improvement_from_previous': 0
        })
        
        print(f"n_estimators={n_est:3d} | R²={r2_score:.4f} | MAE={mae:.4f} | Time={train_time:.1f}s")
    
    # Calculate improvements
    results_df = pd.DataFrame(results)
    results_df['improvement_from_previous'] = results_df['r2_score'].diff()
    results_df['time_increase'] = results_df['train_time_sec'].diff()
    
    print("\n" + "="*70)
    print("ANALYSIS:")
    print("="*70)
    print(results_df.to_string(index=False))
    
    # Find optimal point (best performance/time ratio)
    results_df['efficiency'] = results_df['r2_score'] / results_df['train_time_sec']
    optimal_idx = results_df['efficiency'].idxmax()
    
    print(f"\nMost efficient configuration: n_estimators={results_df.iloc[optimal_idx]['n_estimators']:.0f}")
    print(f"R² Score: {results_df.iloc[optimal_idx]['r2_score']:.4f}")
    print(f"Training Time: {results_df.iloc[optimal_idx]['train_time_sec']:.1f}s")
    
    return results_df


In [57]:
results_df = analyze_n_estimresults_df = analyze_n_estimators_impact(
    X, y_transformed, preprocessor, 
    estimator_range=[50, 100, 200, 300, 500]
)

Analyzing n_estimators impact on RandomForest...
----------------------------------------------------------------------
n_estimators= 50 | R²=0.8042 | MAE=0.7546 | Time=2.3s
n_estimators=100 | R²=0.8027 | MAE=0.7565 | Time=4.1s
n_estimators=200 | R²=0.8033 | MAE=0.7540 | Time=8.6s
n_estimators=300 | R²=0.8030 | MAE=0.7539 | Time=12.8s
n_estimators=500 | R²=0.8025 | MAE=0.7543 | Time=22.7s

ANALYSIS:
 n_estimators  r2_score      mae  train_time_sec  improvement_from_previous  time_increase
           50  0.804226 0.754554        2.338709                        NaN            NaN
          100  0.802676 0.756504        4.133760                  -0.001551       1.795052
          200  0.803317 0.754034        8.592401                   0.000641       4.458640
          300  0.803048 0.753865       12.811828                  -0.000269       4.219427
          500  0.802476 0.754275       22.668892                  -0.000572       9.857064

Most efficient configuration: n_estimators=50
R² S

In [58]:
def find_optimal_complexity(X, y_transformed, preprocessor):
    """
    Find optimal model complexity through learning curves
    Shows when model stops improving with more complexity
    """
    from sklearn.model_selection import learning_curve
    import matplotlib.pyplot as plt
    
    print("Generating learning curves...")
    
    # Test different model complexities
    configs = [
        ('RF-50 trees', RandomForestRegressor(n_estimators=50, n_jobs=-1, random_state=42)),
        ('RF-100 trees', RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42)),
        ('RF-200 trees', RandomForestRegressor(n_estimators=200, n_jobs=-1, random_state=42)),
        ('RF-500 trees (default)', RandomForestRegressor(n_estimators=500, n_jobs=-1, random_state=42)),
    ]
    
    results = []
    for name, model in configs:
        pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('regressor', model)
        ])
        
        # Use smaller cv for speed
        train_sizes, train_scores, val_scores = learning_curve(
            pipeline, X, y_transformed,
            cv=3,  # Reduced for speed
            n_jobs=-1,
            train_sizes=[0.3, 0.6, 0.8, 1.0],
            scoring='r2'
        )
        
        results.append({
            'name': name,
            'train_score_mean': train_scores.mean(axis=1)[-1],
            'val_score_mean': val_scores.mean(axis=1)[-1],
            'val_score_std': val_scores.std(axis=1)[-1]
        })
        
        print(f"{name:25s} | Val R²: {val_scores.mean(axis=1)[-1]:.4f} ± {val_scores.std(axis=1)[-1]:.4f}")
    
    return pd.DataFrame(results)

In [59]:
find_optimal_complexity(X, y_transformed, preprocessor)

Generating learning curves...
RF-50 trees               | Val R²: 0.7422 ± 0.1267
RF-100 trees              | Val R²: 0.7434 ± 0.1260
RF-200 trees              | Val R²: 0.7436 ± 0.1258
RF-500 trees (default)    | Val R²: 0.7444 ± 0.1267


,name,train_score_mean,val_score_mean,val_score_std
0,RF-50 trees,0.969077,0.742218,0.126722
1,RF-100 trees,0.969974,0.743433,0.125967
2,RF-200 trees,0.970822,0.743553,0.125831
3,RF-500 trees (default),0.971042,0.744393,0.126673


In [61]:
def tune_xgboost_randomized(X, y_transformed, preprocessor, n_iter=50):
    """
    Randomized search for XGBoost - faster than GridSearch
    Tests random combinations of parameters
    """
    print("="*70)
    print("TUNING XGBOOST - Randomized Search")
    print("="*70)
    
    # Define parameter distributions
    param_distributions = {
        'regressor__n_estimators': randint(50, 200),  # Random int between 50-200
        'regressor__max_depth': randint(3, 10),       # Depth: 3-10
        'regressor__learning_rate': uniform(0.01, 0.29),  # LR: 0.01-0.30
        'regressor__subsample': uniform(0.6, 0.4),    # Subsample: 0.6-1.0
        'regressor__colsample_bytree': uniform(0.6, 0.4),  # Col sample: 0.6-1.0
        'regressor__min_child_weight': randint(1, 10),
        'regressor__gamma': uniform(0, 0.5),
        'regressor__reg_alpha': uniform(0, 1),  # L1 regularization
        'regressor__reg_lambda': uniform(0, 1)  # L2 regularization
    }
    
    # Create pipeline
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', XGBRegressor(n_jobs=-1, random_state=42))
    ])
    
    # Randomized search
    random_search = RandomizedSearchCV(
        pipeline,
        param_distributions=param_distributions,
        n_iter=n_iter,  # Number of random combinations to try
        cv=5,  # 5-fold CV (faster than 10)
        scoring='neg_mean_absolute_error',  # Optimize for MAE
        n_jobs=-1,
        verbose=2,
        random_state=42,
        return_train_score=True
    )
    
    print(f"\nTesting {n_iter} random parameter combinations...")
    print("This will take approximately 5-15 minutes...\n")
    
    random_search.fit(X, y_transformed)
    
    print("\n" + "="*70)
    print("XGBOOST TUNING RESULTS")
    print("="*70)
    print(f"\nBest Score (MAE): {-random_search.best_score_:.4f}")
    print(f"\nBest Parameters:")
    for param, value in random_search.best_params_.items():
        print(f"  {param.replace('regressor__', '')}: {value}")
    
    # Show top 5 configurations
    results_df = pd.DataFrame(random_search.cv_results_)
    results_df['mean_test_mae'] = -results_df['mean_test_score']
    top_5 = results_df.nsmallest(5, 'mean_test_mae')[
        ['mean_test_mae', 'std_test_score', 'mean_fit_time', 'params']
    ]
    
    print("\nTop 5 Configurations:")
    print(top_5.to_string(index=False))
    
    return random_search.best_estimator_, random_search.best_params_

In [62]:
def tune_lightgbm_randomized(X, y_transformed, preprocessor, n_iter=50):
    """
    Randomized search for LightGBM
    """
    print("="*70)
    print("TUNING LIGHTGBM - Randomized Search")
    print("="*70)
    
    param_distributions = {
        'regressor__n_estimators': randint(50, 200),
        'regressor__max_depth': randint(3, 15),
        'regressor__learning_rate': uniform(0.01, 0.29),
        'regressor__num_leaves': randint(20, 150),
        'regressor__subsample': uniform(0.6, 0.4),
        'regressor__colsample_bytree': uniform(0.6, 0.4),
        'regressor__min_child_samples': randint(10, 50),
        'regressor__reg_alpha': uniform(0, 1),
        'regressor__reg_lambda': uniform(0, 1)
    }
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', lgb.LGBMRegressor(n_jobs=-1, random_state=42, verbose=-1))
    ])
    
    random_search = RandomizedSearchCV(
        pipeline,
        param_distributions=param_distributions,
        n_iter=n_iter,
        cv=5,
        scoring='neg_mean_absolute_error',
        n_jobs=-1,
        verbose=2,
        random_state=42,
        return_train_score=True
    )
    
    print(f"\nTesting {n_iter} random parameter combinations...")
    print("This will take approximately 3-10 minutes...\n")
    
    random_search.fit(X, y_transformed)
    
    print("\n" + "="*70)
    print("LIGHTGBM TUNING RESULTS")
    print("="*70)
    print(f"\nBest Score (MAE): {-random_search.best_score_:.4f}")
    print(f"\nBest Parameters:")
    for param, value in random_search.best_params_.items():
        print(f"  {param.replace('regressor__', '')}: {value}")
    
    return random_search.best_estimator_, random_search.best_params_





    

In [63]:
def evaluate_final_model(best_model, X, y_transformed):
    """
    Comprehensive evaluation of the tuned model
    """
    from sklearn.model_selection import cross_val_score, cross_val_predict
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
    
    print("\n" + "="*70)
    print("FINAL MODEL EVALUATION")
    print("="*70)
    
    # Cross-validation scores
    cv_scores_r2 = cross_val_score(
        best_model, X, y_transformed,
        cv=10,
        scoring='r2',
        n_jobs=-1
    )
    
    cv_scores_mae = cross_val_score(
        best_model, X, y_transformed,
        cv=10,
        scoring='neg_mean_absolute_error',
        n_jobs=-1
    )
    
    print("\nCross-Validation Results (10-fold):")
    print(f"  R² Score:  {cv_scores_r2.mean():.4f} ± {cv_scores_r2.std():.4f}")
    print(f"  MAE:       {-cv_scores_mae.mean():.4f} ± {cv_scores_mae.std():.4f}")
    
    # Get predictions for all data
    y_pred_transformed = cross_val_predict(best_model, X, y_transformed, cv=10)
    
    # Transform back to original scale
    y_pred = np.expm1(y_pred_transformed)
    y_actual = np.expm1(y_transformed)
    
    # Calculate metrics on original scale
    mae = mean_absolute_error(y_actual, y_pred)
    rmse = np.sqrt(mean_squared_error(y_actual, y_pred))
    r2 = r2_score(y_actual, y_pred)
    
    print("\nMetrics on Original Scale (Crores):")
    print(f"  MAE:  {mae:.4f} Crores")
    print(f"  RMSE: {rmse:.4f} Crores")
    print(f"  R²:   {r2:.4f}")
    
    # Percentage errors
    mape = np.mean(np.abs((y_actual - y_pred) / y_actual)) * 100
    print(f"  MAPE: {mape:.2f}%")
    
    return {
        'mae': mae,
        'rmse': rmse,
        'r2': r2,
        'mape': mape
    }


In [64]:
def complete_tuning_workflow(X, y_transformed, preprocessor, method='randomized'):
    """
    Complete hyperparameter tuning workflow
    
    Parameters:
    -----------
    method : str
        'randomized' - Fast, good results (RECOMMENDED)
        'grid' - Thorough, slower
        'sequential' - Easy to understand, moderate speed
    """
    
    print("\n" + "🚀"*35)
    print("HYPERPARAMETER TUNING WORKFLOW")
    print("🚀"*35 + "\n")
    
    # Tune both models
    if method == 'randomized':
        print("\n📊 METHOD: Randomized Search (Fast & Effective)")
        xgb_model, xgb_params = tune_xgboost_randomized(X, y_transformed, preprocessor, n_iter=50)
        lgb_model, lgb_params = tune_lightgbm_randomized(X, y_transformed, preprocessor, n_iter=50)
        
   
        
    
    
    # Evaluate XGBoost
    print("\n" + "="*70)
    print("EVALUATING TUNED XGBOOST")
    xgb_metrics = evaluate_final_model(xgb_model, X, y_transformed)
    
    # Evaluate LightGBM if tuned
    if lgb_model is not None:
        print("\n" + "="*70)
        print("EVALUATING TUNED LIGHTGBM")
        lgb_metrics = evaluate_final_model(lgb_model, X, y_transformed)
        
        # Compare models
        print("\n" + "="*70)
        print("MODEL COMPARISON")
        print("="*70)
        print(f"\nXGBoost  MAE: {xgb_metrics['mae']:.4f}")
        print(f"LightGBM MAE: {lgb_metrics['mae']:.4f}")
        
        if xgb_metrics['mae'] < lgb_metrics['mae']:
            print("\n🏆 WINNER: XGBoost")
            best_model = xgb_model
        else:
            print("\n🏆 WINNER: LightGBM")
            best_model = lgb_model
    else:
        best_model = xgb_model
    
    return best_model, xgb_params


In [65]:
from scipy.stats import randint, uniform, loguniform
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV


In [66]:
best_model, best_params = complete_tuning_workflow(
    X, y_transformed, preprocessor, 
    method='randomized'  
)


🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀
HYPERPARAMETER TUNING WORKFLOW
🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀


📊 METHOD: Randomized Search (Fast & Effective)
TUNING XGBOOST - Randomized Search

Testing 50 random parameter combinations...
This will take approximately 5-15 minutes...

Fitting 5 folds for each of 50 candidates, totalling 250 fits

XGBOOST TUNING RESULTS

Best Score (MAE): 0.1629

Best Parameters:
  colsample_bytree: 0.7509036332911904
  gamma: 0.019481566155116814
  learning_rate: 0.18929362645798892
  max_depth: 9
  min_child_weight: 1
  n_estimators: 139
  reg_alpha: 0.3189756302937613
  reg_lambda: 0.8448753109694546
  subsample: 0.6093087742943303

Top 5 Configurations:
 mean_test_mae  std_test_score  mean_fit_time                                                                                                                                                                                                                                                                       

In [67]:
import pickle

# Save trained pipeline
with open("lightgbm_price_pipeline.pkl", "wb") as f:
    pickle.dump(best_model, f)

print("✅ Pipeline saved successfully!")


✅ Pipeline saved successfully!


In [68]:
with open('df.pkl', 'wb') as file:
    pickle.dump(X, file)

In [69]:
data = [['house', 'sector 102', 4, 3, '3+', 'New Property', 2750, 0, 0, 'unfurnished', 'Low', 'Low Floor']]
columns = ['property_type', 'sector', 'bedRoom', 'bathroom', 'balcony',
       'agePossession', 'built_up_area', 'servant room', 'store room',
       'furnishing_type', 'luxury_category', 'floor_category']

# Convert to DataFrame
one_df = pd.DataFrame(data, columns=columns)

one_df


,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,house,sector 102,4,3,3+,New Property,2750,0,0,unfurnished,Low,Low Floor


In [70]:
np.expm1(best_model.predict(one_df))

array([3.05271557])

In [72]:
len(sorted(X['sector'].unique().tolist()))

355

In [36]:
X[X['sector']==0]

,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
247,0.0,0.0,3,3,2.0,4.0,1267.0,0,0,unfurnished,1.0,2.0
685,0.0,0.0,3,3,4.0,4.0,1940.0,1,0,unfurnished,1.0,2.0
1661,0.0,0.0,3,3,2.0,4.0,1303.0,0,0,unfurnished,1.0,2.0
1822,0.0,0.0,3,3,3.0,1.0,1529.0,0,0,unfurnished,1.0,1.0
2236,0.0,0.0,3,3,3.0,1.0,1357.0,0,0,unfurnished,1.0,2.0
3137,1.0,0.0,5,5,4.0,1.0,600.0,0,0,unfurnished,1.0,2.0
3196,0.0,0.0,3,3,0.0,4.0,1758.0,0,0,unfurnished,1.0,1.0
3464,0.0,0.0,3,4,3.0,4.0,3094.0,0,0,unfurnished,1.0,2.0


In [73]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6049 entries, 0 to 6048
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   property_type    6049 non-null   object 
 1   sector           6049 non-null   object 
 2   bedRoom          6049 non-null   int64  
 3   bathroom         6049 non-null   int64  
 4   balcony          6049 non-null   object 
 5   agePossession    6049 non-null   object 
 6   built_up_area    6049 non-null   float64
 7   servant room     6049 non-null   int64  
 8   store room       6049 non-null   int64  
 9   furnishing_type  6049 non-null   object 
 10  luxury_category  6049 non-null   object 
 11  floor_category   6049 non-null   object 
dtypes: float64(1), int64(4), object(7)
memory usage: 567.2+ KB
